In [102]:
import requests
import json
import pandas as pd
from datetime import datetime
import pytz
from IPython.display import HTML
import time
import datetime

TIMEOUT = 10

In [103]:
#api get call function for incident list
def get_list_of_incidents(tv_host, camera_indices, pattern, username, password):
	url = 'https://' + str(tv_host) + '/json_post'
	
	data = {
		'request': 'cr_get_data_files_info',
		'request_id': 2016,
		'camera_indices': camera_indices,
		'folder': 'incidents',
		'pattern': pattern,
		'host': '127.0.0.1'
	}
	
	try:
		r = requests.post(url, data=json.dumps(data), auth=(username, password), verify=False, timeout=TIMEOUT)
		if r.status_code == requests.codes.ok:
			return r.json()
	except Exception as e:
		return {'error': e}
			
	return {'error': r.status_code}


#api get call function for realtime data
def get_realtime_data(tv_host='', username='', password=''):
	url = 'https://' + str(tv_host) + '/realtime_data'

	try:
		r = requests.get(url, auth=(username,password), verify=False, timeout=TIMEOUT)
		if r.status_code == requests.codes.ok:
			return r.json()
	except Exception as e:
		return {'error': e}

	return {'error': r.status_code}

In [104]:
#actual api call for realtime data from TrafficVision
r = get_realtime_data('tndemo.trafficvision.com:4443', 'HackHPC', 'TeamUser')

#setting data from api call into a dataframe
traffic = pd.DataFrame.from_dict(r['responses'])

#creating separate dataframes for latitude, longitude, and camera_name from realtime_data
cameralat = traffic[['camera_index', 'latitude']].copy()
cameralong = traffic[['camera_index', 'longitude']].copy()
cameraname = traffic[['camera_index', 'camera_name']].copy()

#converting those data frames into dictionaries to add into the incident dataframe
cameralatdict = cameralat.to_dict()
cameralongdict = cameralong.to_dict()
cameranamedict = cameraname.to_dict()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [105]:
#actual api call for 2020 stopped incidents data from TrafficVision
pandas_dict = {'responses': []}    
s = get_list_of_incidents('tndemo.trafficvision.com:4443', list(range(1, 61)), '2020-10*incident_stopped*.json', 'HackHPC', 'TeamUser')
if 'responses' in s:
    for camera_obj in s['responses']:
        camera_index = camera_obj['camera_index']
        incident_entries = camera_obj['entries']
        for entry in incident_entries:
            pandas_dict['responses'].append({'camera_index': camera_index, 'filename': entry['filename'], 'timestamp': entry['time']})

#converting api call data into a dataframe
incidents = pd.DataFrame(pandas_dict['responses'])

#adding columns to incidents from the created dictionaries for latitude, longitude, and camera_name
incidents['Latitude'] = incidents['camera_index'].map(cameralatdict['latitude'])
incidents['Longitude'] = incidents['camera_index'].map(cameralongdict['longitude'])
incidents['CameraName'] = incidents['camera_index'].map(cameranamedict['camera_name'])

#print incidents
incidents.head()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,camera_index,filename,timestamp,Latitude,Longitude,CameraName
0,1,2020-10-02_16-15-27_incident_stopped.json,1601673327,35.0637,-90.024067,"M_I-55 @ US 51, Elvis Presley - Memphis"
1,1,2020-10-05_06-57-41_incident_stopped.json,1601899061,35.0637,-90.024067,"M_I-55 @ US 51, Elvis Presley - Memphis"
2,1,2020-10-03_14-22-38_incident_stopped.json,1601752958,35.0637,-90.024067,"M_I-55 @ US 51, Elvis Presley - Memphis"
3,1,2020-10-05_10-31-19_incident_stopped.json,1601911879,35.0637,-90.024067,"M_I-55 @ US 51, Elvis Presley - Memphis"
4,1,2020-10-07_18-24-27_incident_stopped.json,1602113067,35.0637,-90.024067,"M_I-55 @ US 51, Elvis Presley - Memphis"


In [106]:
#save the DataFrame to a file on the Colab backend.
with open('incidents.json', 'w') as f:
  f.write(incidents.to_json())

In [108]:
#download the file.
#from google.colab import files
#files.download('incidents.json')

In [113]:
#create a dataframe with accumulated values per day
  
incidents['date'] = pd.to_datetime(incidents['timestamp'], unit='s')

incidents['date'] = incidents["date"].dt.strftime("%D")

In [127]:
incidents_table = incidents
incidents_table = incidents_table.drop(['camera_index', 'filename', 'timestamp', 'Latitude', 'Longitude'], axis=1)

valueCount = pd.value_counts(incidents_table['CameraName'])
valueCount = valueCount.head(10)

valueCount


K_I-75 @ Callahan Rd - Knoxville               134
C_I-24 @ S Moore Rd - Chattanooga              123
M_I-55 north of Shelby Dr. - Memphis            82
N_I-40 @ SR 840 JCT () (R3_202) - Nashville     80
N_65 S/O I-840 57.4 - Memphis - Nashville       68
C_I-24 @ Germantown Rd - Chattanooga            65
M_I-55 @ US 51, Elvis Presley - Memphis         61
K_I-75/640 @ Clinton Hwy - Knoxville            60
C_I-24 West of Lookout Creek - Chattanooga      60
K_I-40 @ I-275 - Knoxville                      59
Name: CameraName, dtype: int64

In [147]:
HTML("""
<script src="https://code.highcharts.com/highcharts.js"></script>
<script src="https://code.highcharts.com/modules/heatmap.js"></script>
<script src="https://code.highcharts.com/modules/exporting.js"></script>
<script src="https://code.highcharts.com/modules/export-data.js"></script>
<script src="https://code.highcharts.com/modules/accessibility.js"></script>
<script>
Highcharts.getJSON('Kellentest.json', function (data) {

  <figure class="highcharts-figure">
      <div id="container"></div>
  </figure>
  function getPointCategoryName(point, dimension) {
      var series = point.series,
          isY = dimension === 'y',
          axis = series[isY ? 'yAxis' : 'xAxis'];
      return axis.categories[point[isY ? 'y' : 'x']];
  }

  Highcharts.chart('container', {

      chart: {
          type: 'heatmap',
          marginTop: 40,
          marginBottom: 80,
          plotBorderWidth: 1
      },


      title: {
          text: 'Stopped Incidents in October (Cameras with Highest Incident Rates)'
      },

      xAxis: {
          categories: ['K_I-75 @ Callahan Rd - Knoxville ', 'C_I-24 @ S Moore Rd - Chattanooga', 'M_I-55 north of Shelby Dr. - Memphis', 'N_I-40 @ SR 840 JCT () (R3_202) - Nashville', 'N_65 S/O I-840 57.4 - Memphis - Nashville', 'C_I-24 @ Germantown Rd - Chattanooga', 'M_I-55 @ US 51, Elvis Presley - Memphis', 'K_I-75/640 @ Clinton Hwy - Knoxville', 'C_I-24 West of Lookout Creek - Chattanooga', 'K_I-40 @ I-275 - Knoxville']
      },

      yAxis: {
          categories: ['October'],
          title: null,
          reversed: true
      },

      accessibility: {
          point: {
              descriptionFormatter: function (point) {
                  var ix = point.index + 1,
                      xName = getPointCategoryName(point, 'x'),
                      yName = getPointCategoryName(point, 'y'),
                      val = point.value;
                  return ix + '. ' + xName + ' sales ' + yName + ', ' + val + '.';
              }
          }
      },

      colorAxis: {
          min: 0,
          minColor: '#FFFFFF',
          maxColor: Highcharts.getOptions().colors[0]
      },

      legend: {
          align: 'right',
          layout: 'vertical',
          margin: 0,
          verticalAlign: 'top',
          y: 25,
          symbolHeight: 280
      },

      tooltip: {
          formatter: function () {
              return '<b>' + getPointCategoryName(this.point, 'x') + '</b> registered <br><b>' +
                  this.point.value + '</b> stopped vehicle instances in <br><b>' + getPointCategoryName(this.point, 'y') + '</b>';
          }
      },

      series: [{
          name: 'Sales per employee',
          borderWidth: 1,
          data: data,
          
          dataLabels: {
              enabled: true,
              color: '#000000'
          }
      }],

      responsive: {
          rules: [{
              condition: {
                  maxWidth: 500
              },
              chartOptions: {
                  yAxis: {
                      labels: {
                          formatter: function () {
                              return this.value.charAt(0);
                          }
                      }
                  }
              }
          }]
      }

  }); 
});
</script>
""")